In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator, get_linear_schedule_with_warmup
from peft import get_peft_config, get_peft_model, PromptTuningInit, PromptTuningConfig, TaskType, PeftType, prepare_model_for_int8_training, LoraConfig, get_peft_model_state_dict
import torch
from datasets import load_dataset
import os
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

import transformers
import textwrap
from transformers import LlamaTokenizer, LlamaForCausalLM
import os
import sys
from typing import List
import fire
import torch
from datasets import load_dataset

device = "cuda"
# # model_name_or_path = "bigscience/bloomz-560m"
# # tokenizer_name_or_path = "bigscience/bloomz-560m"
# peft_config = PromptTuningConfig(
#     task_type=TaskType.CAUSAL_LM,
#     prompt_tuning_init=PromptTuningInit.TEXT,
#     num_virtual_tokens=8,
#     tokenizer_name_or_path=model_name_or_path,
# )

# dataset_name = "judge_data"
# checkpoint_name = f"{dataset_name}_{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}_v1.pt".replace(
#     "/", "_"
# )

max_length = 64
lr = 3e-2
num_epochs = 50
batch_size = 8


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /root/anaconda3/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda113.so
CUDA SETUP: CUDA runtime path found: /root/anaconda3/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /root/anaconda3/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...


/root/anaconda3/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/root/anaconda3/lib/libcudart.so.11.0'), PosixPath('/root/anaconda3/lib/libcudart.so')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


In [42]:
import pickle
import json
from pprint import pprint
with open('train_json.pickle', 'rb') as f:
    dataset = pickle.load(f)

train_dataset = dataset["root"]
pprint(train_dataset[0])

print(train_dataset[0]['answer'])


dataset = []
for data in train_dataset:
    a = {
        'instruction' : f"{data['prompt']}",
        'input' : data['input'],
        'output' : data['answer']
    }
    dataset.append(a)

    
print(dataset[0])

with open('alpaca_judge_train.json', 'w') as f:
    json.dump(dataset, f)

{'ID': 'TRAIN_0001',
 'answer': 'yes',
 'input': 'Ramon Nelson was riding his bike when he suffered a lethal blow to '
          'the back of his head with a baseball bat. After two eyewitnesses '
          'identified Lawrence Owens from an array of photos and then a '
          'lineup, he was tried and convicted for Nelson’s death. Because '
          'Nelson was carrying cocaine and crack cocaine potentially for '
          'distribution, the judge at Owens’ bench trial ruled that Owens was '
          'probably also a drug dealer and was trying to “knock [Nelson] off.” '
          'Owens was found guilty of first-degree murder and sentenced to 25 '
          'years in prison.\n'
          'Owens filed a petition for a writ of habeas corpus on the grounds '
          'that his constitutional right to due process was violated during '
          'the trial. He argued that the eyewitness identification should have '
          'been inadmissible based on unreliability and that the judg

In [43]:
pprint(dataset[0])

{'input': 'Ramon Nelson was riding his bike when he suffered a lethal blow to '
          'the back of his head with a baseball bat. After two eyewitnesses '
          'identified Lawrence Owens from an array of photos and then a '
          'lineup, he was tried and convicted for Nelson’s death. Because '
          'Nelson was carrying cocaine and crack cocaine potentially for '
          'distribution, the judge at Owens’ bench trial ruled that Owens was '
          'probably also a drug dealer and was trying to “knock [Nelson] off.” '
          'Owens was found guilty of first-degree murder and sentenced to 25 '
          'years in prison.\n'
          'Owens filed a petition for a writ of habeas corpus on the grounds '
          'that his constitutional right to due process was violated during '
          'the trial. He argued that the eyewitness identification should have '
          'been inadmissible based on unreliability and that the judge '
          'impermissibly inferred a

In [18]:
BASE_MODEL = "decapoda-research/llama-7b-hf"
 
model = LlamaForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
 
tokenizer = LlamaTokenizer.from_pretrained(BASE_MODEL)
 
tokenizer.pad_token_id = (
    0  # unk. we want this to be different from the eos token
)
tokenizer.padding_side = "left"

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


In [19]:
data = load_dataset("json", data_files="alpaca_judge_train.json")
data["train"]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-a50c0b29362ec049/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 2976
})

In [20]:
def generate_prompt(data_point):
    return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.  # noqa: E501
### Instruction:
{data_point["instruction"]}
### Input:
{data_point["input"]}
### Response:
{data_point["output"]}"""
 
 
def tokenize(prompt, add_eos_token=True):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=1024,
        padding=False,
        return_tensors=None,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < 1024
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)
 
    result["labels"] = result["input_ids"].copy()
 
    return result
 
def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenize(full_prompt)
    return tokenized_full_prompt
            
        

In [45]:
generate_prompt(dataset[0])

'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.  # noqa: E501\n### Instruction:\nDoes Lawrence Owens win?\n### Input:\nRamon Nelson was riding his bike when he suffered a lethal blow to the back of his head with a baseball bat. After two eyewitnesses identified Lawrence Owens from an array of photos and then a lineup, he was tried and convicted for Nelson’s death. Because Nelson was carrying cocaine and crack cocaine potentially for distribution, the judge at Owens’ bench trial ruled that Owens was probably also a drug dealer and was trying to “knock [Nelson] off.” Owens was found guilty of first-degree murder and sentenced to 25 years in prison.\nOwens filed a petition for a writ of habeas corpus on the grounds that his constitutional right to due process was violated during the trial. He argued that the eyewitness identification should have been inadmissible based on unrelia

In [21]:

train_val = data["train"].train_test_split(
    test_size=200, shuffle=True, seed=42
)
train_data = (
    train_val["train"].map(generate_and_tokenize_prompt)
)
val_data = (
    train_val["test"].map(generate_and_tokenize_prompt)
)


Map:   0%|          | 0/2776 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [22]:
LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT= 0.05
LORA_TARGET_MODULES = [
    "q_proj",
    "v_proj",
]
 
BATCH_SIZE = 16
MICRO_BATCH_SIZE = 4
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
LEARNING_RATE = 3e-4
TRAIN_STEPS = 300
OUTPUT_DIR = "experiments"

In [23]:
model = prepare_model_for_int8_training(model)
config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=LORA_TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 4194304 || all params: 6742609920 || trainable%: 0.06220594176090199


In [24]:
training_arguments = transformers.TrainingArguments(
    per_device_train_batch_size=MICRO_BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    warmup_steps=100,
    max_steps=TRAIN_STEPS,
    learning_rate=LEARNING_RATE,
    fp16=True,
    logging_steps=10,
    optim="adamw_torch",
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=50,
    save_steps=50,
    output_dir=OUTPUT_DIR,
    save_total_limit=3,
    load_best_model_at_end=True,
    report_to="tensorboard"
)

In [25]:
data_collator = transformers.DataCollatorForSeq2Seq(
    tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
)

In [26]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=training_arguments,
    data_collator=data_collator
)
model.config.use_cache = False
old_state_dict = model.state_dict
model.state_dict = (
    lambda self, *_, **__: get_peft_model_state_dict(
        self, old_state_dict()
    )
).__get__(model, type(model))
 
model = torch.compile(model)
 
trainer.train()
model.save_pretrained(OUTPUT_DIR)

/root/anaconda3/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:321: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss
50,1.348300,1.306234
100,1.209000,1.228881
150,1.195300,1.218913
200,1.188500,1.213241
250,1.144200,1.195231
300,1.155400,1.190804


/root/anaconda3/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:321: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/root/anaconda3/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:321: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/root/anaconda3/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:321: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/root/anaconda3/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:321: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to 

In [27]:
help(get_peft_model_state_dict)

Help on function get_peft_model_state_dict in module peft.utils.save_and_load:

get_peft_model_state_dict(model, state_dict=None, adapter_name='default')
    Get the state dict of the Peft model.
    
    Args:
        model ([`PeftModel`]): The Peft model. When using torch.nn.DistributedDataParallel, DeepSpeed or FSDP,
        the model should be the underlying model/unwrapped model (i.e. model.module).
        state_dict (`dict`, *optional*, defaults to `None`):
            The state dict of the model. If not provided, the state dict of the model
        will be used.



In [28]:
model.eval()

OptimizedModule(
  (_orig_mod): PeftModelForCausalLM(
    (base_model): LoraModel(
      (model): LlamaForCausalLM(
        (model): LlamaModel(
          (embed_tokens): Embedding(32000, 4096, padding_idx=31999)
          (layers): ModuleList(
            (0-31): 32 x LlamaDecoderLayer(
              (self_attn): LlamaAttention(
                (q_proj): Linear8bitLt(
                  in_features=4096, out_features=4096, bias=False
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=4096, out_features=8, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=8, out_features=4096, bias=False)
                  )
                  (lora_embedding_A): ParameterDict()
                  (lora_embedding_B): ParameterDict()
                )
                (k_p

In [36]:
with open('test_json.pickle', 'rb') as f:
    dataset = pickle.load(f)

dataset = dataset["root"]

dataset = [
    {
        'instruction' : x['prompt'],
        'input' : x['input'],
        "ID" : x["ID"],
        "is_first_in" : x["is_first_in"],
        'party' : x['party']
#         'output' : x['answer']
    }
    for x in dataset
]

dataset = sorted(dataset, key=lambda x:x["ID"], reverse=False)
dataset[0]


{'instruction': 'Does Salerno win?',
 'input': 'The 1984 Bail Reform Act allowed the federal courts to detain an arrestee prior to trial if the government could prove that the individual was potentially dangerous to other people in the community. Prosecutors alleged that Salerno and another person in this case were prominent figures in the La Cosa Nostra crime family.\n',
 'ID': 'TEST_0000',
 'is_first_in': 1,
 'party': 'Salerno'}

In [29]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig
from transformers.generation.utils import GreedySearchDecoderOnlyOutput
import textwrap

In [30]:
# def create_prompt(instruction: str) -> str:
#     return PROMPT_TEMPLATE.replace("[INSTRUCTION]", instruction)
def create_prompt(data_point):
    return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.  # noqa: E501
### Instruction:
{data_point["instruction"]}
### Input:
{data_point["input"]}
### Response:
"""



In [31]:
def generate_response(prompt: str, model: model):
    encoding = tokenizer(prompt, return_tensors="pt")
    input_ids = encoding["input_ids"].to(DEVICE)
 
    generation_config = GenerationConfig(
        temperature=0.1,
        top_p=0.75,
        repetition_penalty=1.1,
    )
    with torch.inference_mode():
        return model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=1,
        )

In [32]:
def format_response(response) -> str:
    decoded_output = tokenizer.decode(response.sequences[0])
    response = decoded_output.split("### Response:")[1].strip()
    return "\n".join(textwrap.wrap(response))

In [33]:
def ask_alpaca(prompt, model: model) -> str:
    prompt = create_prompt(prompt)
    response = generate_response(prompt, model)
    response = format_response(response)
    return response

In [34]:
DEVICE='cuda'

In [37]:
from tqdm import tqdm

a = []
i = 0
for inst in tqdm(dataset):
#     print(inst)
    response = ask_alpaca(inst, model)
#     print(response)
    if (response == "yes" and inst['is_first_in'] ==1) or (response == "no" and inst['is_first_in'] ==0) or (inst['party'] == "Complainant" and response == "yes"):
        a.append((inst['ID'], 1))
    else:
        a.append((inst['ID'], 0))  
    
    i += 1

a
    


100%|██████████| 1240/1240 [10:16<00:00,  2.01it/s]


[('TEST_0000', 1),
 ('TEST_0001', 1),
 ('TEST_0002', 1),
 ('TEST_0003', 1),
 ('TEST_0004', 1),
 ('TEST_0005', 1),
 ('TEST_0006', 1),
 ('TEST_0007', 0),
 ('TEST_0008', 0),
 ('TEST_0009', 1),
 ('TEST_0010', 1),
 ('TEST_0011', 1),
 ('TEST_0012', 1),
 ('TEST_0013', 1),
 ('TEST_0014', 1),
 ('TEST_0015', 1),
 ('TEST_0016', 1),
 ('TEST_0017', 0),
 ('TEST_0018', 1),
 ('TEST_0019', 1),
 ('TEST_0020', 1),
 ('TEST_0021', 0),
 ('TEST_0022', 1),
 ('TEST_0023', 1),
 ('TEST_0024', 1),
 ('TEST_0025', 0),
 ('TEST_0026', 1),
 ('TEST_0027', 1),
 ('TEST_0028', 1),
 ('TEST_0029', 1),
 ('TEST_0030', 1),
 ('TEST_0031', 1),
 ('TEST_0032', 1),
 ('TEST_0033', 1),
 ('TEST_0034', 0),
 ('TEST_0035', 1),
 ('TEST_0036', 1),
 ('TEST_0037', 1),
 ('TEST_0038', 0),
 ('TEST_0039', 0),
 ('TEST_0040', 0),
 ('TEST_0041', 1),
 ('TEST_0042', 0),
 ('TEST_0043', 0),
 ('TEST_0044', 1),
 ('TEST_0045', 1),
 ('TEST_0046', 1),
 ('TEST_0047', 1),
 ('TEST_0048', 0),
 ('TEST_0049', 0),
 ('TEST_0050', 0),
 ('TEST_0051', 1),
 ('TEST_0052

In [ ]:
a

In [38]:
import pandas as pd

kk = pd.DataFrame(a, columns=["ID", "first_party_winner"])

kk.to_csv('submit4.csv', index=False)